In [ ]:
import warnings
warnings.simplefilter('ignore', ResourceWarning)
warnings.simplefilter('ignore', UserWarning)
import nilearn
from nilearn import plotting, datasets
import nibabel as nib
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import numpy as np

In [ ]:
def truncate_colormap(cmap, minval=0.0, maxval=1.0, n=100):
    new_cmap = colors.LinearSegmentedColormap.from_list(
        'trunc({n},{a:.2f},{b:.2f})'.format(n=cmap.name, a=minval, b=maxval),
        cmap(np.linspace(minval, maxval, n)))
    return new_cmap

In [ ]:
fsaverage = datasets.fetch_surf_fsaverage('fsaverage5')

In [ ]:
hemis = ['left', 'right']
views = ['posterior']

# Plotting Model Similarity

In [ ]:
cmap = plt.get_cmap('cool')
vis_cmap = truncate_colormap(cmap, 0.15, 0.2)

In [ ]:
!fslmaths brains/vissim36/vissimfish_pearson_func_r2__clusterm_corrp_tstat1.nii.gz -thr 0.95 -bin brains/vissim36/vissim_cluster.nii.gz
!fslmaths brains/vissim36/vissim_cluster.nii.gz -mul brains/vissim36/vissimfish_pearson_func_r2__tstat1.nii.gz \
    brains/vissim36/vissim_thresh.nii.gz

clustMap = nib.load('brains/vissim36/vissim_thresh.nii.gz')

plotting.view_img_on_surf(clustMap, surf_mesh=fsaverage, threshold=0.1, cmap=vis_cmap)



In [ ]:
for hemi in hemis:
    for view in views:
        tex1 = nilearn.surface.vol_to_surf(clustMap, fsaverage['pial_{}'.format(hemi)])

        im = plotting.plot_surf_stat_map(fsaverage['pial_{}'.format(hemi)], tex1, cmap=vis_cmap, 
                                         hemi=hemi, view=view, colorbar=True, 
                                         bg_map=fsaverage['sulc_{}'.format(hemi)], 
                                         threshold=0.1, bg_on_data=True,
                                         output_file='figures/vissim36_{}_{}.pdf'.format(hemi, view))

##  GL code, to be run in MRIcroGL
```python
import gl
ktime= 100

gl.loadimage('mni152')
gl.overlayload('vissim_thresh')
gl.opacity(0,50)
gl.minmax(1, 2.33, 6)
gl.colorname(1,'7cool')
gl.shaderadjust('boundThresh', 0.35)
gl.shaderadjust('edgeThresh', 0.42)
gl.shaderadjust('edgeBoundMix',0.05)
gl.shaderadjust('colorTemp', 0.8)
gl.backcolor(0, 0,0)
gl.clipazimuthelevation(0.36, 0, 0)
gl.shaderadjust('overlayClip', 1)
steps = [0.021, 0.111, 0.201, 0.291, 0.401]
thicks = [0.088, 0.088, 0.088, 0.088, 0.598]
for x, (step, thick) in enumerate(zip(steps, thicks)):
    gl.clipthick(thick)
    gl.clipazimuthelevation(step, 180, 180)
    gl.wait(ktime)
    gl.savebmp(str(x)+'vissim.png')
```

In [ ]:

!cluster --in=brains/vissim36/vissim_thresh.nii.gz \
         --thresh=2.33 \
         --oindex=brains/vissim36/vissim_cluster_index \
         --olmax=brains/vissim36/vissim_lmax.txt \
         --osize=brains/vissim36/vissim_cluster_size \
         --mm


# Plotting NMPH Effect

In [ ]:
cmap = plt.get_cmap('winter')
nmph_cmap = truncate_colormap(cmap, 0.9, 0.95)

In [ ]:
!fslmaths brains/polyfish_pearson_stan_r2__clusterm_corrp_tstat1.nii.gz -thr 0.95 -bin brains/nmph_cluster.nii.gz
!fslmaths brains/nmph_cluster.nii.gz -mul brains/polyfish_pearson_stan_r2__tstat1.nii.gz brains/nmph_thresh.nii.gz

clustMap = nib.load('brains/nmph_thresh.nii.gz')

plotting.view_img_on_surf(clustMap, surf_mesh='fsaverage', threshold=0.1, cmap=nmph_cmap)

In [ ]:
views = ['ventral']

In [ ]:
for hemi in hemis:
    for view in views:
        tex1 = nilearn.surface.vol_to_surf(clustMap, fsaverage['pial_{}'.format(hemi)])

        im = plotting.plot_surf_stat_map(fsaverage['pial_{}'.format(hemi)], tex1, cmap=nmph_cmap, 
                                         hemi=hemi, view=view, colorbar=True, 
                                         bg_map=fsaverage['sulc_{}'.format(hemi)], 
                                         threshold=0.1, bg_on_data=True,
                                         output_file='figures/nmph_{}_{}.pdf'.format(hemi, view))

##  GL code, to be run in MRIcroGL
```python
import gl
ktime= 100

gl.loadimage('mni152')
gl.overlayload('nmph_thresh')
gl.opacity(0,50)
gl.minmax(1, 2.33, 6)
gl.colorname(1,'1red')
gl.shaderadjust('boundThresh', 0.35)
gl.shaderadjust('edgeThresh', 0.42)
gl.shaderadjust('edgeBoundMix',0.05)
gl.shaderadjust('colorTemp', 0.8)
gl.backcolor(0, 0,0)
gl.colorfromzero(1,1)
gl.clipazimuthelevation(0.36, 0, 0)
gl.shaderadjust('overlayClip', 1)
steps = [0.001, 0.151, 0.301, 0.451, 0.601]
thicks = [0.149, 0.149, 0.149, 0.149, 0.398]
for x, (step, thick) in enumerate(zip(steps, thicks)):
    gl.clipthick(thick)
    gl.clipazimuthelevation(step, 180, 180)
    gl.wait(ktime)
    gl.savebmp(str(x)+'nmph.png')
```

In [ ]:

!cluster --in=brains/nmph_thresh.nii.gz \
         --thresh=2.33 \
         --oindex=brains/nmph_cluster_index \
         --olmax=brains/nmph_lmax.txt \
         --osize=brains/nmph_cluster_size \
         --mm
